In [11]:
import numpy as np
import torch
from tqdm import tqdm
from transformers import GPT2LMHeadModel, GPT2TokenizerFast

In [9]:
for model_id in ['gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl']:
    intervals = torch.load(f'wiki_arithmetic_code_{model_id}_test_1.pt')
    print(torch.nonzero(intervals[:,1] - intervals[:,0] <= 0).squeeze())

tensor(98958)
tensor([ 98958, 111097, 205484])
tensor([ 98958, 193427])
tensor(104222)


In [13]:
model_id = 'gpt2'
dataset_split = 'test'
stride = 1
print(f"Computing wiki_arithmetic_code_{model_id}_{dataset_split}_{stride}...")

# --------------------------------------------------------------------------------
# Load the model
# --------------------------------------------------------------------------------

device = "cpu"
tokenizer = GPT2TokenizerFast.from_pretrained(model_id)

# --------------------------------------------------------------------------------
# Load the dataset
# --------------------------------------------------------------------------------

allowed_chars = set("abcdefghijklmnopqrstuvwxyz .,")

dataset_name = "wikitext-103-truncated"
dataset = {}
for item in ["valid", "test"]:
    with open(
        f"{dataset_name}/wiki_shannonfied.{dataset_split}.txt", "r", encoding="utf-8"
    ) as f:
        dataset[dataset_split] = f.read()

encodings = tokenizer(dataset[dataset_split], return_tensors="pt")
encodings.input_ids = encodings.input_ids.to(device)

seq_len = encodings.input_ids.size(1)
print(f"Sequence length = {seq_len}\n")
print(tokenizer.decode(encodings.input_ids[0][seq_len-400:]))

# --------------------------------------------------------------------------------
# Calculate intervals
# --------------------------------------------------------------------------------

def logit_to_cdf(logit):
    prob = np.exp(logit - logit.max())
    cdf = np.cumsum(prob)
    cdf /= cdf.max()
    cdf = np.concatenate((np.zeros(1), cdf))
    return cdf

def logit_array_to_cdf(logit, axis, epsilon=1e-9):
    if isinstance(logit, np.ndarray):
        logit = torch.tensor(logit)

    max_logit = torch.max(logit, axis=axis, keepdims=True)[0]
    prob = torch.exp(logit - max_logit)
    prob /= prob.sum(axis=axis, keepdims=True)
    prob += epsilon
    cdf = torch.cumsum(prob, axis=axis)
    cdf /= torch.max(cdf, axis=axis, keepdims=True)[0]
    # append 0 to the beginning of the cdf along axis=axis
    shape = list(cdf.shape)
    shape[axis] = 1
    cdf = torch.concatenate((torch.zeros(shape).to(cdf.device), cdf), axis=axis)
    return cdf

def get_intervals(logits, symbols, epsilon=1e-9):
    original_shape = logits.shape
    logits = logits.reshape(-1, logits.shape[-1])
    symbols = symbols.reshape(-1)

    cdf = logit_array_to_cdf(logits, axis=1, epsilon=epsilon)

    intervals = []
    for i, symbol in enumerate(symbols):
        lower_bound = cdf[i, symbol]
        upper_bound = cdf[i, symbol + 1]

        intervals.append([lower_bound.item(), upper_bound.item()])

    # Convert intervals list to a tensor
    intervals_tensor = torch.tensor(intervals)
    intervals_tensor = intervals_tensor.reshape(original_shape[:-1] + (2,))
    return intervals_tensor

Computing wiki_arithmetic_code_gpt2_test_1...


Token indices sequence length is longer than the specified maximum sequence length for this model (258499 > 1024). Running this sequence through the model will result in indexing errors


Sequence length = 258499

 lavishly praised for their performances and the film is summed up as not to be missed. paul newman reprised his role as fast eddie felson in the film the color of money, for which he won the academy award for best actor in a leading role. a number of observers and critics have suggested that this oscar was in belated recognition for his performance in the hustler. in, the library of congress selected the hustler for preservation in the united states national film registry as culturally, historically, or aesthetically significant. carroll and rossen s screenplay was selected by the writers guild of america in as the th best motion picture screenplay of all time. in june, afi released its ten top ten the best ten films in ten classic american film genres after polling over, people from the creative community. the hustler was acknowledged as the sixth best film in the sports genre. the hustler is credited with sparking a resurgence in the popularity of pool in t

In [66]:
model_id = 'gpt2'
model = GPT2LMHeadModel.from_pretrained(model_id).to(device)
max_length = model.config.n_positions
seq_len = encodings.input_ids.size(1)

intervals_list = []
bad_index = 98958
for begin_loc in tqdm(range(bad_index, bad_index+1, stride)):
    end_loc = min(begin_loc + max_length, seq_len)
    if end_loc + 1 >= seq_len:
        break # Let's just throw away the tail. It's easier than dealing with the annoying tail.
    input_ids = encodings.input_ids[:, begin_loc:end_loc]
    with torch.no_grad():
        outputs = model(input_ids)

    logits = outputs.logits[0, -stride:]
    symbols = encodings.input_ids[:, end_loc+1-stride:end_loc+1]
    print(symbols)
    intervals_list.append(get_intervals(logits, symbols, epsilon=1e-7))

bad_intervals = torch.cat(intervals_list, axis=0)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.84s/it]

tensor([[27870]])


In [49]:
print(logits)
print(encodings.input_ids[:, bad_index])
print(logits.exp()[0,284])

tensor([[-237.1587, -234.2314, -237.7939,  ..., -249.6300, -248.3404,
         -235.4836]])
tensor([284])
tensor(0.)


In [63]:
bugged_logits = torch.load('debug_gpt2_test_1.pt', map_location=torch.device('cpu'))
print(logits - bugged_logits)
epsilon = 1e-7

cdf = logit_array_to_cdf(logits, axis=1, epsilon=epsilon)
bugged_cdf = logit_array_to_cdf(bugged_logits, axis=1, epsilon=epsilon)
print((cdf - bugged_cdf).max())
bad_id = 284
print(cdf[0,bad_id+1] - cdf[0,bad_id])
print(bugged_cdf[0,bad_id+1] - bugged_cdf[0,bad_id])

tensor([[-1.6785e-04, -4.5776e-05,  1.5259e-05,  ..., -6.1035e-05,
         -7.6294e-05,  1.2207e-04]])
tensor(1.1921e-07)
tensor(5.3854e-07)
tensor(5.3842e-07)


In [70]:
epsilon = 1e-7
print(get_intervals(bugged_logits, torch.tensor([27870]), epsilon)[0,1] - get_intervals(bugged_logits, torch.tensor([27870]), epsilon)[0,0])

tensor(1.1921e-07)


In [32]:
%pdb on
for model_id in ['gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl']:
    intervals = torch.load(f'wiki_arithmetic_code_{model_id}_test_1.pt')
    print(intervals[bad_index-3:bad_index+3,1] - intervals[bad_index-3:bad_index+3,0])
    print(intervals[bad_index-3:bad_index+3] - bad_intervals)

Automatic pdb calling has been turned ON
tensor([ 5.6726e-04,  8.4885e-01,  5.4737e-02, -1.1921e-07,  3.8313e-01,
         2.3260e-01])
tensor([[-2.9802e-07, -3.5763e-07],
        [ 5.1782e-07, -1.3113e-06],
        [ 4.6492e-06,  2.5630e-06],
        [ 0.0000e+00, -2.3842e-07],
        [ 1.4007e-06, -1.7881e-06],
        [-8.9034e-07, -2.2948e-06]])
tensor([ 5.1636e-04,  8.6871e-01,  3.9888e-02, -1.1921e-07,  1.3010e-02,
         2.0468e-01])
tensor([[-0.1110, -0.1110],
        [-0.0063,  0.0135],
        [-0.0438, -0.0587],
        [-0.0007, -0.0007],
        [ 0.1695, -0.2006],
        [-0.0101, -0.0380]])
tensor([ 4.5690e-04,  9.4215e-01,  2.0052e-03, -1.1921e-07,  3.8215e-01,
         4.1126e-01])
tensor([[-0.1529, -0.1530],
        [-0.0108,  0.0825],
        [-0.0433, -0.0960],
        [-0.0014, -0.0014],
        [-0.0290, -0.0299],
        [-0.0258,  0.1529]])
tensor([1.1526e-03, 9.6889e-01, 4.2533e-03, 4.1723e-07, 2.1789e-01, 2.6212e-01])
tensor([[-0.0395, -0.0389],
        [-